In [1]:
import re
import numpy as np
import pandas as pd
import feather
import xgboost as xgb
import feather

from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, confusion_matrix, f1_score, accuracy_score

from utils import print_confusion_matrix

/home/watts/anaconda2/envs/aind-dog/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df_train = np.load('../cache/train_stage1_fe2.npy')

df1 = pd.read_csv('../cache/stage1_labels.csv')
y = df1['y'].values

df_test = np.load('../cache/test_stage1_fe2.npy')
df = pd.read_csv('../cache/stage1_test_id.csv')
pid = df.ID.values

In [3]:
df_test.shape

(5668, 10483)

In [4]:
df_train.shape

(3321, 10483)

In [5]:
y = y - 1 #fix for zero bound array

In [6]:
wts_per_class = np.load('../cache/stage1_train_weights_per_class.npy')
wts_per_class = wts_per_class.tolist()

In [7]:
print(wts_per_class)

{1: 4.846830985915493, 2: 6.347345132743363, 3: 36.31460674157304, 4: 3.8411078717201166, 5: 12.723140495867769, 6: 11.076363636363636, 7: 2.484784889821616, 8: 173.78947368421052, 9: 88.75675675675676}


In [8]:
 
-

In [9]:
submission = pd.DataFrame(preds/denom, columns=['class'+str(c+1) for c in range(9)])
submission['ID'] = pid
# submission.to_csv('../submissions/sub_stage1_all_2_2_xgb.csv', index=False)
submission.to_csv('../submissions/sub_stage1_all_2_2_wt_xgb.csv', index=False)

In [10]:
# score 0.4571 on stage1 public leader board

In [11]:
df_train = np.load('../cache/train_p_stage1_fe2.npy')

df1 = pd.read_csv('../cache/stage1_labels.csv')
y = df1['y'].values

df_test = np.load('../cache/test_p_stage1_fe2.npy')
df = pd.read_csv('../cache/stage1_p_test_id.csv')
pid = df.ID.values

In [12]:
print(df_train.shape, df_test.shape)


(3321, 10233) (368, 10233)


In [13]:
df = pd.read_csv('../cache/stage1_p_test_labels.csv')
test_labels = df['y'].values-1

In [14]:
test_labels.shape

(368,)

In [15]:
y = y - 1 #fix for zero bound array

In [19]:
denom = 0
fold = 10 
for i in range(fold):
    params = {
        'eta': 0.03333,
        'max_depth': 6,
        'subsample' : 0.8,
        'colsample_bytree':0.8,
        'objective': 'multi:softprob',
        'eval_metric': 'mlogloss',
        'num_class': 9,
        'seed': i,
        'silent': True
    }
    x1, x2, y1, y2 = train_test_split(df_train, y, test_size=0.2, random_state=i)
    w1 = [wts_per_class[j+1] for j in y1]
    w2 = [wts_per_class[j+1] for j in y2]
    watchlist = [(xgb.DMatrix(x1, y1,weight=w1), 'train'), (xgb.DMatrix(x2, y2,weight=w2), 'valid')]
    model = xgb.train(params, xgb.DMatrix(x1, y1,weight=w1), 1000,  watchlist, verbose_eval=50, early_stopping_rounds=100)
    score1 = log_loss(y2, model.predict(xgb.DMatrix(x2), 
                                        ntree_limit=model.best_ntree_limit),  
                      labels = list(range(9)))
    print('fold = {:d}'.format(i))
    print('val multi_log_loss: {}'.format(score1))
    
    #if score < 0.9:
    if denom != 0:
        pred = model.predict(xgb.DMatrix(df_test), ntree_limit=model.best_ntree_limit+80)
        preds += pred
    else:
        pred = model.predict(xgb.DMatrix(df_test), ntree_limit=model.best_ntree_limit+80)
        preds = pred.copy()
    
    denom += 1
    score2 = log_loss(test_labels, pred, labels = list(range(9)))
    print('test multi_log_loss: {}'.format(score2))
    
    fscore = f1_score(test_labels, pred.argmax(axis=1), labels = list(range(9)), average='macro')
    print('test f1_score: {}'.format(fscore))
    
    acc = accuracy_score(test_labels, pred.argmax(axis=1))
    print('test accuracy: {}'.format(acc))
    
    print(confusion_matrix(test_labels, pred.argmax(axis=1), labels = list(range(9))))
    
    print('-------------------')
    print('\n\n')
    
#     submission = pd.DataFrame(pred, columns=['class'+str(c+1) for c in range(9)])
#     submission['ID'] = pid
#     submission.to_csv('../submissions/sub_stage1_2_2_wt_xgb_fold_'  + str(i) + '.csv', index=False)

[0]	train-mlogloss:2.10856	valid-mlogloss:2.14116
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 100 rounds.
[50]	train-mlogloss:0.649044	valid-mlogloss:1.31363
[100]	train-mlogloss:0.307304	valid-mlogloss:1.19975
[150]	train-mlogloss:0.177905	valid-mlogloss:1.20502
[200]	train-mlogloss:0.111166	valid-mlogloss:1.2464
Stopping. Best iteration:
[121]	train-mlogloss:0.241751	valid-mlogloss:1.18755

fold = 0
val multi_log_loss: 1.0166732667196066
test multi_log_loss: 1.2472289983770284
test f1_score: 0.3422667306312961
test accuracy: 0.529891304347826
[[53  2  1 21 12  2  3  0  0]
 [ 4 14  0  3  0  3 22  0  0]
 [ 0  0  2  2  0  1  2  0  0]
 [17  0  1 42  2  0  3  0  0]
 [ 3  0  0  6  7  6  3  0  0]
 [ 2  1  0  2  5  6  6  0  0]
 [ 3 16  0  7  1  3 71  0  0]
 [ 0  1  0  0  0  0  1  0  0]
 [ 2  0  0  3  0  0  1  0  0]]
-------------------





/home/watts/anaconda2/envs/aind-dog/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[0]	train-mlogloss:2.11198	valid-mlogloss:2.14265
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 100 rounds.
[50]	train-mlogloss:0.656255	valid-mlogloss:1.38142
[100]	train-mlogloss:0.31457	valid-mlogloss:1.30404
[150]	train-mlogloss:0.185324	valid-mlogloss:1.34003
Stopping. Best iteration:
[91]	train-mlogloss:0.352454	valid-mlogloss:1.29816

fold = 1
val multi_log_loss: 1.0484916906159623
test multi_log_loss: 1.2944071755704025
test f1_score: 0.33961538623616216
test accuracy: 0.5027173913043478
[[52  2  0 22 13  4  1  0  0]
 [ 3  9  1  4  0  2 27  0  0]
 [ 0  0  2  2  0  2  1  0  0]
 [20  1  1 36  1  0  6  0  0]
 [ 0  0  1  5 13  2  4  0  0]
 [ 3  1  3  1  4  2  8  0  0]
 [ 2 21  2  4  0  2 70  0  0]
 [ 0  1  0  0  0  0  1  0  0]
 [ 2  0  0  2  0  0  1  0  1]]
-------------------



[0]	train-mlogloss:2.11042	valid-mlogloss:2.1419
Multiple eval metrics have been passed: 'valid-mlogloss' wil

In [20]:
score2 = log_loss(test_labels, pred, labels = list(range(9)))
print('test multi_log_loss: {}'.format(score2))

fscore = f1_score(test_labels, pred.argmax(axis=1), labels = list(range(9)), average='macro')
print('test f1_score: {}'.format(fscore))

acc = accuracy_score(test_labels, pred.argmax(axis=1))
print('test accuracy: {}'.format(acc))

print(confusion_matrix(test_labels, pred.argmax(axis=1), labels = list(range(9))))

# with no lambda, no alpha we get
# val multi_log_loss: 1.0247328862547875
# test multi_log_loss: 1.41869956478679
# test f1_score: 0.2549920698056178
# test accuracy: 0.4429347826086957

# with alpha: 100 we get:
# val multi_log_loss: 1.1108881736468328
# test multi_log_loss: 1.5692192874848843
# test f1_score: 0.27029986646550364
# test accuracy: 0.40217391304347827

# with alpha 500 we get:
# test multi_log_loss: 1.9443259673274083
# test f1_score: 0.2713471712458167
# test accuracy: 0.30978260869565216
    
# below with lambda 700
# test multi_log_loss: 1.45

test multi_log_loss: 1.2932468328888163
test f1_score: 0.43393423412464877
test accuracy: 0.5380434782608695
[[52  0  1 18 17  2  4  0  0]
 [ 4 12  0  3  0  1 26  0  0]
 [ 1  0  1  1  0  0  4  0  0]
 [18  1  1 39  0  0  6  0  0]
 [ 0  0  1  7 14  1  2  0  0]
 [ 1  1  1  4  1  7  7  0  0]
 [ 6 11  2 10  2  1 69  0  0]
 [ 0  0  0  1  0  0  1  0  0]
 [ 0  0  0  2  0  0  0  0  4]]


/home/watts/anaconda2/envs/aind-dog/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [21]:
pred1 = preds/denom

score2 = log_loss(test_labels, pred1, labels = list(range(9)))
print('final multi_log_loss: {}'.format(score2))

fscore = f1_score(test_labels, pred1.argmax(axis=1), labels = list(range(9)), average='macro')
print('final f1_score: {}'.format(fscore))

acc = accuracy_score(test_labels, pred1.argmax(axis=1))
print('final accuracy: {}'.format(acc))

print(confusion_matrix(test_labels, pred1.argmax(axis=1), labels = list(range(9))))

final multi_log_loss: 1.2342428618070225
final f1_score: 0.36933577459893246
final accuracy: 0.5652173913043478
[[57  2  1 18 13  1  2  0  0]
 [ 3 11  0  4  0  1 27  0  0]
 [ 1  0  2  1  0  1  2  0  0]
 [17  1  1 42  0  0  4  0  0]
 [ 0  0  0  6 16  1  2  0  0]
 [ 2  1  1  3  3  5  7  0  0]
 [ 4 12  2  6  0  2 75  0  0]
 [ 0  1  0  0  0  0  1  0  0]
 [ 2  0  0  3  0  0  1  0  0]]


/home/watts/anaconda2/envs/aind-dog/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
